In [139]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from copy import deepcopy
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc

In [140]:
# node class that each spot in the map will occupy
# cell location and goal_location are tuples representing index 
# of current cell location and goal cell locations
class Node:
    def __init__(self, parent, c2c, is_obstacle, cell_location):
        self.parent = parent
        self.c2c = c2c
        self.is_obstacle = is_obstacle
        self.cell_location = cell_location

In [141]:
def check_obstacle(x, y):
    
#     if x >= 260 and x <= 340:
#         if y <= np.sqrt(40**2 - (x-300)^2) + 185:
#             return True

    # check circle
    if y <= 225 and y >= 145 and x <= np.sqrt(40**2 - (y-185)**2) + 300 and x >= -np.sqrt(40**2 - (y-185)**2) + 300:
        return True
    
    # check triangle
    

    
    
    return False

In [142]:
def check_margin(x, y):
    return False

In [143]:
# create the board

def create_board(width, height):

    board = []
    for row_num in range(0, height):
        temp_row = []
        for col_num in range(0, width):

            c2c = np.Infinity
            
            is_obstacle = check_obstacle(col_num, row_num)
#             is_obstacle = False

            new_node = Node(parent=None, c2c=c2c, is_obstacle=is_obstacle, cell_location=(row_num, col_num))
            temp_row.append(new_node)

        board.append(temp_row)

    return board

In [144]:
def create_color_map(board):

    color_map = np.zeros(shape=[250, 400, 3], dtype=np.uint8)

    for row in range(250):
        for col in range(400):
            
            if board[row][col].is_obstacle:
                color_map[row][col][0] = 255
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0
            else:
                color_map[row][col][0] = 0
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0
    
    return color_map


In [145]:
def update_color_map(curr_node, color_map,  brg_color):

    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]

    color_map[row][col][0] = brg_color[0]
    color_map[row][col][1] = brg_color[1]
    color_map[row][col][2] = brg_color[2]
    return color_map


In [146]:
def get_start_node(x, y, board):
    if (x >= 0 and x < 400) and (y >= 0 and y < 250):
        if not board[x][y].is_obstacle:
            return board[x][y]
        else:
            print('Board Location is in an obstacle. Choose another location')
            return
            
    print('Board Location Out of bounds. Choose another Location')
    return

In [160]:
def check_up(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if row < 249:
        
        # check if obstacle
        new_node = board[row+1][col]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [161]:
def check_down(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if row > 0:
        
        # check if obstacle
        new_node = board[row-1][col]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [162]:
def check_left(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col > 0:
        
        # check if obstacle
        new_node = board[row][col-1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [163]:
def check_right(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col < 399:
        
        # check if obstacle
        new_node = board[row][col+1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [164]:
def check_up_left(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col > 0 and row < 249:
        
        # check if obstacle
        new_node = board[row+1][col-1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1.4 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [165]:
def check_up_right(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col < 399 and row < 249:
        
        # check if obstacle
        new_node = board[row+1][col+1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1.4 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [166]:
def check_down_left(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col > 0 and row > 0:
        
        # check if obstacle
        new_node = board[row-1][col-1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1.4 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [168]:
def check_down_right(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if col < 399 and row > 0:
        
        # check if obstacle
        new_node = board[row-1][col+1]
        
        if not new_node.is_obstacle:
            
            new_c2c = 1.4 + curr_node.c2c
            
            if new_c2c <= new_node.c2c:
                print(f'Updating c2c and parent at {new_node.cell_location}')
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [169]:
def gen_next_nodes(board, curr_node):

    new_nodes = []

    new_nodes.append(check_up(board, curr_node))
    new_nodes.append(check_down(board, curr_node))
    new_nodes.append(check_left(board, curr_node))
    new_nodes.append(check_right(board, curr_node))
    new_nodes.append(check_up_left(board, curr_node))
    new_nodes.append(check_up_right(board, curr_node))
    new_nodes.append(check_down_left(board, curr_node))
    new_nodes.append(check_down_right(board, curr_node))

    return list(filter(lambda node: node is not None, new_nodes))

In [170]:
def get_solution_path(curr_node):
    solution_path= []
    
    while curr_node:
        solution_path.insert(0, curr_node)
        curr_node = curr_node.parent
        
    return solution_path

In [171]:
def animate(closed_nodes, solution_path, filename):
    out = cv2.VideoWriter(f'{filename}.avi',cv2.VideoWriter_fourcc(*'DIVX'), 60, (400, 250))
 
    for node in closed_nodes:
        out.write(np.flipud(update_color_map(node, color_map, [255, 255, 255])))

    for node in solution_path:
        out.write(np.flipud(update_color_map(node, color_map, [0, 255, 0])))
        
    out.release()

In [177]:
# construct the graph of nodes as well as an array of blank rbg values that will represent the board
# perform disjkstra using the node grah, but update the color array as we pop nodes

start_location = (0, 0)
width = 400
height = 250
goal_location = (100, 100)

# create the starting board and color map based off of the starting board
board = create_board(width, height)
color_map = create_color_map(board)

start_node = get_start_node(start_location[0], start_location[1], board)
start_node.c2c = 0

open_nodes = [start_node]
closed_nodes = []

while len(open_nodes) > 0:
    # generate the colors of the current board and append it to the list
    # this will be a frame of an animation
    # color_maps.append(gen_color_map(board)
    open_nodes.sort(key=lambda x: x.c2c)
    curr_node = open_nodes.pop(0)
    closed_nodes.append(curr_node)


    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    print(f"Searching ({row},{col})")

    if curr_node.cell_location == goal_location:
        print('Found Solution')
        break
    else:
        next_possible_nodes = gen_next_nodes(board, curr_node)
        for node in next_possible_nodes:

            appendable = True

            for o_node in open_nodes:
                if o_node == node:
                    appendable = False
                    break
            if appendable:
                for c_node in closed_nodes:
                    if c_node == node:
                        appendable = False
                        break

            if appendable:
                open_nodes.append(node)



Searching (0,0)
Updating c2c and parent at (1, 0)
Updating c2c and parent at (0, 1)
Updating c2c and parent at (1, 1)
Searching (1,0)
Updating c2c and parent at (2, 0)
Updating c2c and parent at (2, 1)
Searching (0,1)
Updating c2c and parent at (0, 2)
Updating c2c and parent at (1, 2)
Searching (1,1)
Updating c2c and parent at (2, 2)
Searching (2,0)
Updating c2c and parent at (3, 0)
Updating c2c and parent at (3, 1)
Searching (0,2)
Updating c2c and parent at (0, 3)
Updating c2c and parent at (1, 3)
Searching (2,1)
Updating c2c and parent at (3, 2)
Searching (1,2)
Updating c2c and parent at (2, 3)
Searching (2,2)
Updating c2c and parent at (3, 3)
Searching (3,0)
Updating c2c and parent at (4, 0)
Updating c2c and parent at (4, 1)
Searching (0,3)
Updating c2c and parent at (0, 4)
Updating c2c and parent at (1, 4)
Searching (3,1)
Updating c2c and parent at (4, 2)
Searching (1,3)
Updating c2c and parent at (2, 4)
Searching (3,2)
Updating c2c and parent at (4, 3)
Searching (2,3)
Updating c2c

Searching (21,19)
Updating c2c and parent at (22, 20)
Searching (19,21)
Updating c2c and parent at (20, 22)
Searching (25,9)
Updating c2c and parent at (26, 10)
Searching (9,25)
Updating c2c and parent at (10, 26)
Searching (27,4)
Updating c2c and parent at (28, 5)
Searching (4,27)
Updating c2c and parent at (5, 28)
Searching (22,17)
Updating c2c and parent at (23, 18)
Searching (17,22)
Updating c2c and parent at (18, 23)
Searching (24,12)
Updating c2c and parent at (25, 13)
Searching (12,24)
Updating c2c and parent at (13, 25)
Searching (26,7)
Updating c2c and parent at (27, 8)
Searching (7,26)
Updating c2c and parent at (8, 27)
Searching (28,2)
Updating c2c and parent at (29, 3)
Searching (2,28)
Updating c2c and parent at (3, 29)
Searching (23,15)
Updating c2c and parent at (24, 16)
Searching (15,23)
Updating c2c and parent at (16, 24)
Searching (25,10)
Updating c2c and parent at (26, 11)
Searching (10,25)
Updating c2c and parent at (11, 26)
Searching (21,20)
Updating c2c and parent 

Searching (14,32)
Updating c2c and parent at (15, 33)
Searching (34,9)
Updating c2c and parent at (35, 10)
Searching (9,34)
Updating c2c and parent at (10, 35)
Searching (36,4)
Updating c2c and parent at (37, 5)
Searching (4,36)
Updating c2c and parent at (5, 37)
Searching (29,22)
Updating c2c and parent at (30, 23)
Searching (22,29)
Updating c2c and parent at (23, 30)
Searching (31,17)
Updating c2c and parent at (32, 18)
Searching (17,31)
Updating c2c and parent at (18, 32)
Searching (27,27)
Updating c2c and parent at (28, 28)
Searching (33,12)
Updating c2c and parent at (34, 13)
Searching (12,33)
Updating c2c and parent at (13, 34)
Searching (35,7)
Updating c2c and parent at (36, 8)
Searching (7,35)
Updating c2c and parent at (8, 36)
Searching (37,2)
Updating c2c and parent at (38, 3)
Searching (2,37)
Updating c2c and parent at (3, 38)
Searching (30,20)
Updating c2c and parent at (31, 21)
Searching (20,30)
Updating c2c and parent at (21, 31)
Searching (28,25)
Updating c2c and parent 

Updating c2c and parent at (5, 43)
Searching (33,27)
Updating c2c and parent at (34, 28)
Searching (27,33)
Updating c2c and parent at (28, 34)
Searching (35,22)
Updating c2c and parent at (36, 23)
Searching (22,35)
Updating c2c and parent at (23, 36)
Searching (37,17)
Updating c2c and parent at (38, 18)
Searching (17,37)
Updating c2c and parent at (18, 38)
Searching (39,12)
Updating c2c and parent at (40, 13)
Searching (12,39)
Updating c2c and parent at (13, 40)
Searching (41,7)
Updating c2c and parent at (42, 8)
Searching (7,41)
Updating c2c and parent at (8, 42)
Searching (43,2)
Updating c2c and parent at (44, 3)
Searching (2,43)
Updating c2c and parent at (3, 44)
Searching (34,25)
Updating c2c and parent at (35, 26)
Searching (25,34)
Updating c2c and parent at (26, 35)
Searching (32,30)
Updating c2c and parent at (33, 31)
Searching (30,32)
Updating c2c and parent at (31, 33)
Searching (36,20)
Updating c2c and parent at (37, 21)
Searching (20,36)
Updating c2c and parent at (21, 37)
S

Updating c2c and parent at (21, 41)
Searching (42,15)
Updating c2c and parent at (43, 16)
Searching (15,42)
Updating c2c and parent at (16, 43)
Searching (44,10)
Updating c2c and parent at (45, 11)
Searching (10,44)
Updating c2c and parent at (11, 45)
Searching (46,5)
Updating c2c and parent at (47, 6)
Searching (5,46)
Updating c2c and parent at (6, 47)
Searching (48,0)
Updating c2c and parent at (49, 0)
Updating c2c and parent at (49, 1)
Searching (0,48)
Updating c2c and parent at (0, 49)
Updating c2c and parent at (1, 49)
Searching (37,28)
Updating c2c and parent at (38, 29)
Searching (28,37)
Updating c2c and parent at (29, 38)
Searching (35,33)
Updating c2c and parent at (36, 34)
Searching (33,35)
Updating c2c and parent at (34, 36)
Searching (39,23)
Updating c2c and parent at (40, 24)
Searching (23,39)
Updating c2c and parent at (24, 40)
Searching (41,18)
Updating c2c and parent at (42, 19)
Searching (18,41)
Updating c2c and parent at (19, 42)
Searching (43,13)
Updating c2c and par

...

Searching (40,121)
Updating c2c and parent at (40, 122)
Updating c2c and parent at (41, 122)
Searching (119,45)
Updating c2c and parent at (120, 45)
Updating c2c and parent at (120, 46)
Searching (45,119)
Updating c2c and parent at (45, 120)
Updating c2c and parent at (46, 120)
Searching (123,35)
Updating c2c and parent at (124, 35)
Updating c2c and parent at (124, 36)
Searching (35,123)
Updating c2c and parent at (35, 124)
Updating c2c and parent at (36, 124)
Searching (125,30)
Updating c2c and parent at (126, 30)
Updating c2c and parent at (126, 31)
Searching (30,125)
Updating c2c and parent at (30, 126)
Updating c2c and parent at (31, 126)
Searching (117,50)
Updating c2c and parent at (118, 51)
Searching (50,117)
Updating c2c and parent at (51, 118)
Searching (127,25)
Updating c2c and parent at (128, 26)
Searching (25,127)
Updating c2c and parent at (26, 128)
Searching (129,20)
Updating c2c and parent at (130, 21)
Searching (20,129)
Updating c2c and parent at (21, 130)
Searching (13

Searching (59,114)
Updating c2c and parent at (60, 115)
Searching (112,64)
Updating c2c and parent at (113, 65)
Searching (64,112)
Updating c2c and parent at (65, 113)
Searching (110,69)
Updating c2c and parent at (111, 70)
Searching (69,110)
Updating c2c and parent at (70, 111)
Searching (108,74)
Updating c2c and parent at (109, 75)
Searching (74,108)
Updating c2c and parent at (75, 109)
Searching (106,79)
Updating c2c and parent at (107, 80)
Searching (79,106)
Updating c2c and parent at (80, 107)
Searching (104,84)
Updating c2c and parent at (105, 85)
Searching (84,104)
Updating c2c and parent at (85, 105)
Searching (102,89)
Updating c2c and parent at (103, 90)
Searching (89,102)
Updating c2c and parent at (90, 103)
Searching (100,94)
Updating c2c and parent at (101, 95)
Searching (94,100)
Updating c2c and parent at (95, 101)
Searching (121,42)
Updating c2c and parent at (122, 43)
Searching (42,121)
Updating c2c and parent at (43, 122)
Searching (119,47)
Updating c2c and parent at (1

Searching (11,134)
Updating c2c and parent at (11, 135)
Updating c2c and parent at (12, 135)
Searching (136,6)
Updating c2c and parent at (137, 6)
Updating c2c and parent at (137, 7)
Searching (6,136)
Updating c2c and parent at (6, 137)
Updating c2c and parent at (7, 137)
Searching (116,56)
Updating c2c and parent at (117, 57)
Searching (56,116)
Updating c2c and parent at (57, 117)
Searching (138,1)
Updating c2c and parent at (139, 2)
Searching (1,138)
Updating c2c and parent at (2, 139)
Searching (114,61)
Updating c2c and parent at (115, 62)
Searching (61,114)
Updating c2c and parent at (62, 115)
Searching (112,66)
Updating c2c and parent at (113, 67)
Searching (66,112)
Updating c2c and parent at (67, 113)
Searching (110,71)
Updating c2c and parent at (111, 72)
Searching (71,110)
Updating c2c and parent at (72, 111)
Searching (108,76)
Updating c2c and parent at (109, 77)
Searching (76,108)
Updating c2c and parent at (77, 109)
Searching (106,81)
Updating c2c and parent at (107, 82)
Sea

Searching (80,107)
Updating c2c and parent at (81, 108)
Searching (105,85)
Updating c2c and parent at (106, 86)
Searching (85,105)
Updating c2c and parent at (86, 106)
Searching (103,90)
Updating c2c and parent at (104, 91)
Searching (90,103)
Updating c2c and parent at (91, 104)
Searching (101,95)
Updating c2c and parent at (102, 96)
Searching (95,101)
Updating c2c and parent at (96, 102)
Searching (122,43)
Updating c2c and parent at (123, 44)
Searching (43,122)
Updating c2c and parent at (44, 123)
Searching (120,48)
Updating c2c and parent at (121, 49)
Searching (48,120)
Updating c2c and parent at (49, 121)
Searching (124,38)
Updating c2c and parent at (125, 39)
Searching (38,124)
Updating c2c and parent at (39, 125)
Searching (126,33)
Updating c2c and parent at (127, 34)
Searching (33,126)
Updating c2c and parent at (34, 127)
Searching (128,28)
Updating c2c and parent at (129, 29)
Searching (28,128)
Updating c2c and parent at (29, 129)
Searching (130,23)
Updating c2c and parent at (1

Searching (57,117)
Updating c2c and parent at (58, 118)
Searching (139,2)
Updating c2c and parent at (140, 3)
Searching (2,139)
Updating c2c and parent at (3, 140)
Searching (115,62)
Updating c2c and parent at (116, 63)
Searching (62,115)
Updating c2c and parent at (63, 116)
Searching (113,67)
Updating c2c and parent at (114, 68)
Searching (67,113)
Updating c2c and parent at (68, 114)
Searching (111,72)
Updating c2c and parent at (112, 73)
Searching (72,111)
Updating c2c and parent at (73, 112)
Searching (109,77)
Updating c2c and parent at (110, 78)
Searching (77,109)
Updating c2c and parent at (78, 110)
Searching (107,82)
Updating c2c and parent at (108, 83)
Searching (82,107)
Updating c2c and parent at (83, 108)
Searching (105,87)
Updating c2c and parent at (106, 88)
Searching (87,105)
Updating c2c and parent at (88, 106)
Searching (103,92)
Updating c2c and parent at (104, 93)
Searching (92,103)
Updating c2c and parent at (93, 104)
Searching (101,97)
Updating c2c and parent at (102, 

In [178]:
solution_path = get_solution_path(curr_node)
animate(closed_nodes, solution_path, filename='project')
